In [1]:
_RIOT_API_KEY='RGAPI-0539714d-a04d-4e39-acb8-138983063a28'
from riotwatcher import LolWatcher
import helper_functions as help
import role_classification as rc
import constants
import pandas as pd
from datetime import datetime
import mysql.connector
from sqlalchemy import create_engine
import concurrent.futures
import analysis_functions as af

lol_watcher = LolWatcher(_RIOT_API_KEY)

1. Implement retry for when the URL gets too many requests for 1 match. probably as simple as a wrapper function around the slurps in slurp_data that retries if the clienter error exception is returned
e.g. Exception 429 Client Error: Too Many Requests for url: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4708472978/timeline thrown. Cancelling futures.

2. Handle this exception: Exception 404 Client Error: Not Found for url: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_3891774034 thrown. Cancelling futures.

3. Handling processing by puuid instead of summonerName? 
    GoofyGriefer	cT70cPTfqJ_gLJrZRqSTBO0lO3ZfM25tP4MRfvVsPw5CF5EHcdPILgojfnvBMudR9vJmnNAtFxiBvg
    CerebralPlayer	cT70cPTfqJ_gLJrZRqSTBO0lO3ZfM25tP4MRfvVsPw5CF5EHcdPILgojfnvBMudR9vJmnNAtFxiBvg
    PepegaMindset	cT70cPTfqJ_gLJrZRqSTBO0lO3ZfM25tP4MRfvVsPw5CF5EHcdPILgojfnvBMudR9vJmnNAtFxiBvg

In [2]:
engine = help.create_mysql_engine()

In [65]:
s_n='souporsecret'
rol='ALL ROLES'
query = f"""
select distinct 
	summonername,
    count(distinct matchid)
from lol_match_details
where matchid in ( select distinct matchid from lol_match_details where summonername = '{s_n}') and summonername !='{s_n}'
group by 1
order by 2 desc
limit 20;
"""
friends = pd.read_sql(query, con=engine)
friends

,summonername,count(distinct matchid)
0,IrishGuy,642
1,AerialBlast,541
2,SpicedCider,488
3,radaemon1,464
4,koreazian,160
5,Teroboal,118
6,Kroocs,54
7,imnaeatchu,46
8,JellyBlaster,36
9,LIAAAAAAAAAAAAAA,20


In [85]:
#list_of_friends = ['radaemon1','irishguy','spicedcider','aerialblast','teroboal','koreazian']
list_of_friends = [i.lower() for i in list(friends.summonername)]

d= {s_n:rol}


temp_df = af.stats_at_min_with_teammates(s_n, rol, engine, d, return_type='normal')
temp_df=temp_df[temp_df.Champion_Name=='Total']
temp_df.rename(columns={'Champion_Name':'Friend_Name'}, inplace=True)
temp_df.reset_index(inplace=True,drop=True)
temp_df.loc[0, 'Friend_Name']=s_n
for friend in list_of_friends:
    team_dict= {s_n:rol, friend:'ALL ROLES'}
    friend_df = af.stats_at_min_with_teammates(s_n, rol, engine, team_dict, return_type='normal')
    friend_df.rename(columns={'Champion_Name':'Friend_Name'}, inplace=True)
    friend_df.reset_index(inplace=True, drop=True)
    friend_df.loc[0, 'Friend_Name']=friend
    temp_df.loc[len(temp_df)]=friend_df.loc[0]
temp_df.sort_values(by='Games',ascending=False, inplace=True)
temp_df.reset_index(drop=True, inplace=True)

c:\Users\mmend\OneDrive\riot_api_git_clone_folder\Riot_Api\v1\analysis_functions.py:121: RuntimeWarning: invalid value encountered in scalar divide
  total_row = ['Total', n_games,  aggregated_df.Games.dot(aggregated_df.Winrate)/n_games,
c:\Users\mmend\OneDrive\riot_api_git_clone_folder\Riot_Api\v1\analysis_functions.py:122: RuntimeWarning: invalid value encountered in scalar divide
  aggregated_df.Games.dot(aggregated_df['CS_Diff@14'])/n_games,
c:\Users\mmend\OneDrive\riot_api_git_clone_folder\Riot_Api\v1\analysis_functions.py:123: RuntimeWarning: invalid value encountered in scalar divide
  aggregated_df.Games.dot(aggregated_df['XP_Diff@14'])/n_games,
c:\Users\mmend\OneDrive\riot_api_git_clone_folder\Riot_Api\v1\analysis_functions.py:124: RuntimeWarning: invalid value encountered in scalar divide
  aggregated_df.Games.dot(aggregated_df['Gold_Diff@14'])/n_games,
c:\Users\mmend\OneDrive\riot_api_git_clone_folder\Riot_Api\v1\analysis_functions.py:125: RuntimeWarning: invalid value encou

In [84]:
format_dict={'Game': "{:.0f}",
                'Winrate': "{:.0%}",
                'CS_Diff@14': "{:.0f}",
                'XP_Diff@14': "{:.0f}",
                'Gold_Diff@14': "{:.0f}",
                'DMG_Diff@14': "{:.0f}"}
temp_df.style.format(format_dict)


,index,Friend_Name,Games,Winrate,CS_Diff@14,XP_Diff@14,Gold_Diff@14,DMG_Diff@14
0,0,souporsecret,760,53%,-0,-0,107,746
1,1,irishguy,424,54%,0,52,141,853
2,3,spicedcider,364,52%,-1,13,151,882
3,2,aerialblast,359,51%,-1,-8,109,783
4,4,radaemon1,311,56%,-1,9,135,729
5,5,koreazian,86,58%,-1,5,214,746
6,6,teroboal,44,59%,-2,66,125,684
7,9,jellyblaster,35,46%,-4,-247,-355,493
8,8,imnaeatchu,28,46%,-0,49,291,1396
9,7,kroocs,14,50%,-1,60,28,399


In [74]:
# ~25 seconds for 50 matches. More if we've hit our API Limit.
# 8m 16s for 800 records

# help.collect_riot_api_data('SouporSecret', lol_watcher, 'lol_match_timeline',
#                             engine, match_upload_limit=800, num_worker_threads=2)

In [6]:
# 23.7 seconds for 50 matches, 1 thread
# 7.4 seconds for 20 matches, 2 threads
# 5 seconds for 20 matches, 4 threads
# 4.7 seconds for 20 matches, 8 threads
# 5.8 seconds for 50 matches, 8 threads

# help.collect_riot_api_data('SouporSecret', lol_watcher, 'lol_match_details',
#                             engine, match_upload_limit=1200, num_worker_threads=2)


In [13]:
help.collect_riot_api_data('ireliathinkhard', lol_watcher, 'lol_match_timeline',
                        engine, match_upload_limit=40, num_worker_threads=4)

935 matches to slurp!
Uploaded match id NA1_4749140412 lol_match_timeline data.
Uploaded match id NA1_4747376829 lol_match_timeline data.
Uploaded match id NA1_4749102247 lol_match_timeline data.
Uploaded match id NA1_4747553897 lol_match_timeline data.
Uploaded match id NA1_4741898748 lol_match_timeline data.
Uploaded match id NA1_4740981877 lol_match_timeline data.
Uploaded match id NA1_4739909906 lol_match_timeline data.
Uploaded match id NA1_4740760887 lol_match_timeline data.
Uploaded match id NA1_4739934695 lol_match_timeline data.
Uploaded match id NA1_4738335359 lol_match_timeline data.
Uploaded match id NA1_4738783055 lol_match_timeline data.
Uploaded match id NA1_4739823906 lol_match_timeline data.
Uploaded match id NA1_4738268063 lol_match_timeline data.
Uploaded match id NA1_4738289066 lol_match_timeline data.
Uploaded match id NA1_4737326218 lol_match_timeline data.Uploaded match id NA1_4737863776 lol_match_timeline data.

Uploaded match id NA1_4733600127 lol_match_timelin

In [12]:
help.collect_riot_api_data('ireliathinkhard', lol_watcher, 'lol_match_details',
                        engine, match_upload_limit=40, num_worker_threads=4)

935 matches to slurp!
Uploaded match id NA1_4749140412 lol_match_details data.
Uploaded match id NA1_4749102247 lol_match_details data.
Uploaded match id NA1_4747553897 lol_match_details data.
Uploaded match id NA1_4747376829 lol_match_details data.
Uploaded match id NA1_4741898748 lol_match_details data.
Uploaded match id NA1_4739934695 lol_match_details data.
Uploaded match id NA1_4740760887 lol_match_details data.
Uploaded match id NA1_4740981877 lol_match_details data.
Uploaded match id NA1_4739909906 lol_match_details data.
Uploaded match id NA1_4738783055 lol_match_details data.
Uploaded match id NA1_4739823906 lol_match_details data.
Uploaded match id NA1_4738335359 lol_match_details data.
Uploaded match id NA1_4738289066 lol_match_details data.
Uploaded match id NA1_4737863776 lol_match_details data.
Uploaded match id NA1_4738268063 lol_match_details data.
Uploaded match id NA1_4737306861 lol_match_details data.
Uploaded match id NA1_4730411246 lol_match_details data.
Uploaded 

In [14]:
rc.determine_roles(engine)

41
0 matches with more than 2 smites:
0 matches with more than 2 champs with support items:
41 match roles uploaded, including: 
['NA1_4010105744' 'NA1_4749140412' 'NA1_4749102247' 'NA1_4747553897'
 'NA1_4747376829' 'NA1_4741898748' 'NA1_4739934695' 'NA1_4740760887'
 'NA1_4740981877' 'NA1_4739909906']


In [5]:
list_of_friends = ['radaemon1','koreazian','SouporSecret','SpicedCider','AerialBlast','IrishGuy',
'GoofyGriefer','LIAAAAAAAAAAAAAA','Kroocs','JellyBlaster','MegaJharo','Teroboal','Boss 0f This Gym']
for friend in list_of_friends:
    print(f"Fetching data for {friend}!")
    help.collect_riot_api_data(friend, lol_watcher, 'lol_match_timeline',
                            engine, match_upload_limit=1200, num_worker_threads=4)
    help.collect_riot_api_data(friend, lol_watcher, 'lol_match_details',
                            engine, match_upload_limit=1200, num_worker_threads=4)
rc.determine_roles(engine)

Fetching data for radaemon1!
0 matches to slurp!
0 matches to slurp!
1
Not enough roles to classify
Fetching data for koreazian!
0 matches to slurp!
0 matches to slurp!
1
Not enough roles to classify
Fetching data for SouporSecret!
1 matches to slurp!
Empty Game. Returning None for Match Timeline.
Match id NA1_4393904069 returns None.
1 matches to slurp!
Empty Game. Returning None for Match Details.
Match id NA1_4393904069 returns None.
1
Not enough roles to classify
Fetching data for SpicedCider!
1 matches to slurp!
Empty Game. Returning None for Match Timeline.
Match id NA1_4393904069 returns None.
2 matches to slurp!
Empty Game. Returning None for Match Details.
Match id NA1_4393904069 returns None.
Empty Game. Returning None for Match Details.
Match id NA1_4235418390 returns None.
1
Not enough roles to classify
Fetching data for AerialBlast!
3 matches to slurp!
Empty Game. Returning None for Match Timeline.
Match id NA1_4393904069 returns None.
Exception 404 Client Error: Not Found

c:\Users\mmend\OneDrive\riot_api_git_clone_folder\Riot_Api\v1\analysis_functions.py:111: RuntimeWarning: invalid value encountered in scalar divide
  total_row = ['Total', n_games,  aggregated_df.Games.dot(aggregated_df.Winrate)/n_games,
c:\Users\mmend\OneDrive\riot_api_git_clone_folder\Riot_Api\v1\analysis_functions.py:112: RuntimeWarning: invalid value encountered in scalar divide
  aggregated_df.Games.dot(aggregated_df['CS_Diff@14'])/n_games,
c:\Users\mmend\OneDrive\riot_api_git_clone_folder\Riot_Api\v1\analysis_functions.py:113: RuntimeWarning: invalid value encountered in scalar divide
  aggregated_df.Games.dot(aggregated_df['XP_Diff@14'])/n_games,
c:\Users\mmend\OneDrive\riot_api_git_clone_folder\Riot_Api\v1\analysis_functions.py:114: RuntimeWarning: invalid value encountered in scalar divide
  aggregated_df.Games.dot(aggregated_df['Gold_Diff@14'])/n_games,
c:\Users\mmend\OneDrive\riot_api_git_clone_folder\Riot_Api\v1\analysis_functions.py:115: RuntimeWarning: invalid value encou